<a href="https://colab.research.google.com/github/sayanbanerjee32/multimodal_llm/blob/main/phi_3_QLoRA_instruct150k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers==4.45.2
!pip install -Uq accelerate peft bitsandbytes trl dataset
# transformers
# !pip install -Uq flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
import numpy as np
import copy
import requests
from tqdm import tqdm
import os, gc
import subprocess
import json
import random
### Download Phi-3 model
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, PreTrainedModel
from transformers.trainer_callback import TrainerCallback
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
import joblib
from huggingface_hub import hf_hub_download

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from peft.tuners import lora
from huggingface_hub import HfApi

from bitsandbytes.nn import LinearFP4
from huggingface_hub import hf_hub_download
import torch.cuda.amp as amp
import dataclasses

In [ ]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Download Phi-3 model

In [ ]:
# Load the Phi-3 model and tokenizer
model_name = "microsoft/Phi-3.5-mini-instruct"
# "microsoft/Phi-3-mini-4k-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right",
#                                            trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

### Downlaod image embedding

In [ ]:
# URL of the embeddings file (replace with your actual URL)
embeddings_url = '/content/drive/MyDrive/multimodal_llm/image_embedding/coco_image_embeddings.npz'

# Load the embeddings
print("Loading embeddings...")
embeddings = np.load(embeddings_url, allow_pickle=True)

# Print embeddings and image names
for image_name, embedding in embeddings.items():
    print(f"Image: {image_name}")
    print(f"Embedding shape: {embedding.shape}")
    print(f"Embedding preview: {embedding[:5]}...")  # Print first 5 values
    print("-" * 50)
    break

print(f"Total number of embeddings: {len(embeddings)}")

Loading embeddings...
Image: 000000401144.jpg
Embedding shape: (512,)
Embedding preview: [-0.13     0.1564   0.02017  0.1678   0.2393 ]...
--------------------------------------------------
Total number of embeddings: 81479


### Data processing

In [ ]:
# List of URLs to download
url = "https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json"

# Download each file
subprocess.run(["wget", "-c", url])

# Load the downloaded JSON file
json_file = "llava_instruct_150k.json"
with open(json_file, 'r') as f:
    data = json.load(f)


In [ ]:
# data = data[0:10000]

In [ ]:
def create_dataset():
    processed_data = []
    print("Processing data...")
    with tqdm(total=len(data)) as pbar:
        for item in data:
            image_file = item['image']
            if image_file in embeddings:
                processed_data.append({
                    'image': image_file,
                    'image_embedding': embeddings[image_file].tolist(),
                    'conversation': item['conversations']
                })
            pbar.update(1)

    print(f"Data processing completed. Total processed items: {len(processed_data)}")

    return Dataset.from_dict({
        "image": [item['image'] for item in processed_data],
        "image_embedding": [item['image_embedding'] for item in processed_data],
        "conversation": [item['conversation'] for item in processed_data]
    })

print("Creating HuggingFace dataset...")
hf_dataset = create_dataset()

print("HuggingFace dataset creation completed.")
print(f"Total samples in dataset: {len(hf_dataset)}")



Creating HuggingFace dataset...
Processing data...


100%|██████████| 10000/10000 [00:57<00:00, 173.03it/s]


Data processing completed. Total processed items: 10000
HuggingFace dataset creation completed.
Total samples in dataset: 10000


In [ ]:
def prepare_dataset(examples):
    image_embeddings = []
    conversations = []

    for idx, conv in enumerate(examples['conversation']):
        image_embedding = torch.tensor(examples['image_embedding'][idx])
        dialogue_pairs = []

        for i in range(0, len(conv), 2):
            if i + 1 < len(conv):  # Ensure we have a pair
                human_msg = conv[i]['value'].replace('<image>', '').strip()
                gpt_msg = conv[i + 1]['value']

                dialogue = [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Given the following information, provide a detailed and accurate response:\n{human_msg}\n[An image is provided for this task.]\n"},
                    {"role": "assistant", "content": gpt_msg}
                ]

                dialogue_pairs.append(dialogue)
                image_embeddings.append(image_embedding)

        conversations.extend(dialogue_pairs)

    image_embeddings = torch.stack(image_embeddings)

    tokenized_conversations = tokenizer.apply_chat_template(conversations,
                                                            return_tensors='pt', padding=True)

    return {
        "image_embeddings": image_embeddings,
        "input_ids": tokenized_conversations,
        "attention_mask": torch.ones_like(tokenized_conversations),
        "labels": tokenized_conversations.clone()
    }

# Test the prepare_dataset function with a real training example
def test_prepare_dataset():
    # Get a batch of examples from the dataset
    batch_size = 1  # You can adjust this as needed
    sample_batch = hf_dataset[5:5+batch_size]

    print("Original conversations:")
    # for i, sample in enumerate(sample_batch):
    #     print(f"\nSample {i + 1}:")
    for message in sample_batch['conversation'][0]:
        print(f"{message['from']}: {message['value']}")

    # Process the sample batch
    result = prepare_dataset(sample_batch)

    # Print the structure of the result
    print("\nResult keys:", result.keys())
    print("Image embeddings shape:", result['image_embeddings'].shape)
    print("Input IDs shape:", result['input_ids'].shape)
    print("Attention mask shape:", result['attention_mask'].shape)
    print("Labels shape:", result['labels'].shape)

    for i in range(batch_size):
        decoded_input = tokenizer.decode(result['input_ids'][i])
        decoded_labels = tokenizer.decode(result['labels'][i])

        print(f"\nRestructured input for sample {i + 1}:")
        print(decoded_input)

        print(f"\nLabels for sample {i + 1}:")
        print(decoded_labels)

        # Optionally, you can print a more readable version of the labels
        print("\nReadable labels (non-padding tokens):")
        readable_labels = tokenizer.decode([token for token in result['labels'][i] if token != -100])
        print(readable_labels)

    # Optionally, you can print attention mask to see where it's applied
    print("\nAttention Mask:")
    print(result['attention_mask'][0])
# Run the test
test_prepare_dataset()


Original conversations:
human: <image>
What is the girl eating in the image?
gpt: The girl in the image is eating a dessert, which appears to be a graham cracker treat or a cookie sandwich.
human: Describe the girl's hair color and clothing.
gpt: The girl has blonde hair, and she is wearing a pink shirt.
human: What color is the plate that the dessert is on?
gpt: The dessert is on a green plate.
human: Is the girl looking at the camera or focusing on her dessert?
gpt: The girl is looking up at the camera while taking a bite of her dessert.
human: Where is the girl eating her dessert?
gpt: The girl is eating her dessert at the table.

Result keys: dict_keys(['image_embeddings', 'input_ids', 'attention_mask', 'labels'])
Image embeddings shape: torch.Size([5, 512])
Input IDs shape: torch.Size([5, 75])
Attention mask shape: torch.Size([5, 75])
Labels shape: torch.Size([5, 75])

Restructured input for sample 1:
<|system|> You are a helpful assistant.<|end|><|user|> Given the following infor

In [ ]:
# Apply tokenization and prepare the dataset
print("Applying tokenization and preparing the dataset...")

hf_dataset_mapped = hf_dataset.map(
    prepare_dataset,
    batched=True,
    remove_columns=hf_dataset.column_names,
    batch_size=1024  # Adjust based on your memory constraints
).with_format("torch")

# Split the dataset
train_test_split = hf_dataset_mapped.train_test_split(test_size=0.05)

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

print(f"Train dataset size: {len(dataset_dict['train'])}")
print(f"Test dataset size: {len(dataset_dict['test'])}")

Applying tokenization and preparing the dataset...


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Train dataset size: 43134
Test dataset size: 2271


In [ ]:
# Example of accessing an item:
sample = dataset_dict['train'][0]
print(f"Input IDs shape: {len(sample['input_ids'])}")
print(f"Attention mask shape: {len(sample['attention_mask'])}")
print(f"Labels shape: {len(sample['labels'])}")

Input IDs shape: 673
Attention mask shape: 673
Labels shape: 673


### Projection Layer

### QLoRA set up

In [ ]:
# new_model = "ms-phi3-custom"
lora_r = 16 #32 #64
lora_alpha = 16
lora_dropout = 0.05
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./multimodal-phi3_5-mini-instruct-llava_adapter"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 8
per_device_eval_batch_size = 4
gradient_accumulation_steps = 4
gradient_checkpointing = True
max_grad_norm = 1.0 #0.3
learning_rate = 5e-4
weight_decay = 0.0 #0.001
optim = "adamw_torch" #"paged_adamw_32bit"
lr_scheduler_type = "linear" #"constant"
max_steps = -1
warmup_ratio = 0.1 #0.03
group_by_length = True
save_steps = 25
logging_steps = 25
eval_steps = 5 # Evaluate every 25 steps
max_seq_length = 256
packing = False
device_map = {"": 0}
hf_adapter_repo="sayanbanerjee32/multimodal-phi3_5-mini-instruct-llava_adapter"

In [ ]:
class ImageProjector(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=1024):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.GELU()
        self.layer2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.05)

        # Store initial weights for both layers
        self.register_buffer('initial_weights1', self.layer1.weight.data.clone())
        self.register_buffer('initial_weights2', self.layer2.weight.data.clone())

    def forward(self, x):
        # # Print dtypes
        # print(f"Input dtype: {x.dtype}")
        # print(f"Layer1 weight dtype: {self.layer1.weight.dtype}")
        # print(f"Layer1 bias dtype: {self.layer1.bias.dtype}")
        # print(f"Layer2 weight dtype: {self.layer2.weight.dtype}")
        # print(f"Layer2 bias dtype: {self.layer2.bias.dtype}")

        x = self.layer1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x

    def get_weight_change(self):
        current_weights1 = self.layer1.weight.data
        current_weights2 = self.layer2.weight.data

        # Ensure all tensors are on the same device
        device = current_weights1.device
        initial_weights1 = self.initial_weights1.to(device)
        initial_weights2 = self.initial_weights2.to(device)

        weight_diff1 = torch.norm(current_weights1 - initial_weights1).item()
        weight_diff2 = torch.norm(current_weights2 - initial_weights2).item()
        return weight_diff1 + weight_diff2  # Total weight change across both layers

class Phi3WithProjector(PreTrainedModel):
    supports_gradient_checkpointing = True
    _supports_sdpa = True # Add this line

    def __init__(self, phi3_model, projector, debug=False):
        super().__init__(phi3_model.config)
        self.phi3 = phi3_model
        self.projector = projector
        self.debug = debug

    def debug_print(self, *args, **kwargs):
        if self.debug:
            print(*args, **kwargs)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, *model_args, debug=False, **kwargs):
        # Load the base Phi-3 model
        phi3_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path, *model_args, **kwargs)

        # Determine if it's a local path or a Hugging Face model ID
        is_local = os.path.isdir(pretrained_model_name_or_path)

        if is_local:
            projector_path = os.path.join(pretrained_model_name_or_path, "image_projector.pth")
        else:
            try:
                # Try to download the projector weights from the Hugging Face Hub
                projector_path = hf_hub_download(repo_id=pretrained_model_name_or_path, filename="image_projector.pth")
            except Exception as e:
                print(f"Failed to download projector weights: {e}")
                projector_path = None

        if projector_path and os.path.exists(projector_path):
            projector_state_dict = torch.load(projector_path, map_location=phi3_model.device)
            # Check if the state dict has the expected structure
            if 'linear.weight' in projector_state_dict:
                input_dim = projector_state_dict['linear.weight'].size(1)
                output_dim = projector_state_dict['linear.weight'].size(0)
            else:
                # If not, try to infer dimensions from the first layer's weight
                first_key = next(iter(projector_state_dict))
                input_dim = projector_state_dict[first_key].size(1)
                output_dim = phi3_model.config.hidden_size  # Assuming this is the correct output dimension

            projector = ImageProjector(input_dim, output_dim)

            # # Convert projector weights and biases to the same dtype as the main model
            # target_dtype = kwargs.get('torch_dtype', torch.float32)
            # projector_state_dict = {k: v.to(target_dtype) for k, v in projector_state_dict.items()}

            # Load the state dict with converted weights and biases
            projector.load_state_dict(projector_state_dict, strict=False)

            # # Ensure all parameters (including biases) are in the correct dtype
            # for param in projector.parameters():
            #     param.data = param.data.to(target_dtype)

            print(f"Loaded projector with input_dim={input_dim}, output_dim={output_dim}")#, dtype={target_dtype}")
        else:
            print(f"Projector weights not found. Initializing with default dimensions.")
            input_dim = 512  # Default CLIP embedding size
            output_dim = phi3_model.config.hidden_size
            # target_dtype = kwargs.get('torch_dtype', torch.float32)
            projector = ImageProjector(input_dim, output_dim)
            # Ensure all parameters (including biases) are in the correct dtype
            # for param in projector.parameters():
            #     param.data = param.data.to(target_dtype)

        # Move the projector to the same device as phi3_model
        projector = projector.to(phi3_model.device)

        # Create and return the Phi3WithProjector instance
        model = cls(phi3_model, projector, debug=debug)
        return model

    def save_pretrained(self, save_directory):
        print(f"Saving model to {save_directory}")

        # Save the base model
        self.phi3.save_pretrained(save_directory)

        # Save the projector weights
        projector_path = os.path.join(save_directory, "image_projector.pth")
        projector_state = self.projector.state_dict()
        print(f"Projector weights stats before saving:")
        for name, param in projector_state.items():
            print(f"  {name}: mean={param.mean().item():.4f}, std={param.std().item():.4f}")
        torch.save(projector_state, projector_path)

        # Save the config
        self.config.save_pretrained(save_directory)

        print(f"Model saved successfully to {save_directory}")

    def forward(self, input_ids=None, attention_mask=None, labels=None, image_embeddings=None, past_key_values=None, **kwargs):
        with amp.autocast():
            device = next(self.parameters()).device

            if image_embeddings is not None:
                projected_embeddings = self.projector(image_embeddings).to(torch.float16)
                # Ensure projected_embeddings requires grad
                if not projected_embeddings.requires_grad:
                    projected_embeddings.requires_grad_(True)
                projected_embeddings = projected_embeddings.unsqueeze(1)
                self.debug_print(f"forward projected_embeddings: {projected_embeddings.size()}")

                if past_key_values is None:  # This is the first forward pass
                    self.debug_print(f"forward before: {attention_mask.size() if attention_mask is not None else None}")
                    if 'inputs_embeds' in kwargs and kwargs['inputs_embeds'] is not None:
                        inputs_embeds = kwargs['inputs_embeds']
                        self.debug_print(f"forward before inputs_embeds: {inputs_embeds.size()}")
                        inputs_embeds = torch.cat([projected_embeddings, inputs_embeds], dim=1)
                        kwargs['inputs_embeds'] = inputs_embeds
                        self.debug_print(f"forward after inputs_embeds: {inputs_embeds.size()}")
                    elif input_ids is not None:
                        self.debug_print(f"forward input_ids: {input_ids.size()}")
                        inputs_embeds = self.get_input_embeddings()(input_ids.to(device))
                        self.debug_print(f"forward before inputs_embeds: {inputs_embeds.size()}")
                        inputs_embeds = torch.cat([projected_embeddings, inputs_embeds], dim=1)
                        self.debug_print(f"forward after inputs_embeds: {inputs_embeds.size()}")
                        kwargs['inputs_embeds'] = inputs_embeds
                        input_ids = None  # Set to None to avoid conflict

                    if attention_mask is not None:
                        attention_mask = torch.cat([torch.ones(image_embeddings.size(0), 1, device=device), attention_mask.to(device)], dim=1)
                    else:
                        attention_mask = torch.ones(image_embeddings.size(0), inputs_embeds.size(1), device=device)

                    if labels is not None:
                        # Adjust labels to match the new sequence length
                        labels = torch.cat([torch.full((labels.size(0), 1), -100, device=device), labels], dim=1)

            if labels is not None:
                labels = labels.to(device)

            # Determine sequence length
            if 'inputs_embeds' in kwargs and kwargs['inputs_embeds'] is not None:
                seq_length = kwargs['inputs_embeds'].size(1)
            elif input_ids is not None:
                seq_length = input_ids.size(1)
            else:
                seq_length = attention_mask.size(1) if attention_mask is not None else None

            if seq_length is None:
                raise ValueError("Unable to determine sequence length. Provide either input_ids, inputs_embeds, or attention_mask.")

            # Ensure attention_mask matches the sequence length
            if attention_mask is not None:
                attention_mask = attention_mask[:, :seq_length]

            self.debug_print(f"forward final: input_ids shape: {input_ids.shape if input_ids is not None else None}")
            self.debug_print(f"forward final: attention_mask shape: {attention_mask.shape if attention_mask is not None else None}")
            self.debug_print(f"forward final: inputs_embeds shape: {kwargs.get('inputs_embeds', {}).shape if kwargs.get('inputs_embeds') is not None else None}")

            return self.phi3(input_ids=input_ids, attention_mask=attention_mask, labels=labels, past_key_values=past_key_values, **kwargs)

    def prepare_inputs_for_generation(self, input_ids, past=None, attention_mask=None, **kwargs):
        inputs = self.phi3.prepare_inputs_for_generation(input_ids, past=past, attention_mask=attention_mask, **kwargs)

        if 'image_embeddings' in kwargs:
            inputs['image_embeddings'] = kwargs['image_embeddings']

            if past is None:  # First forward pass
                # Adjust attention_mask to account for the image token
                if attention_mask is not None:
                    inputs['attention_mask'] = torch.cat([torch.ones((attention_mask.size(0), 1), device=attention_mask.device), attention_mask], dim=1)
            else:  # Subsequent passes
                # Ensure attention_mask matches the current sequence length
                if attention_mask is not None:
                    current_seq_length = past[0][0].size(2) + 1  # past key's sequence length + 1 for the new token
                    inputs['attention_mask'] = attention_mask[:, :current_seq_length]

            inputs.pop('position_ids', None)

        # Safe printing of shapes
        self.debug_print(f"prepare_inputs_for_generation: input_ids shape: {inputs['input_ids'].shape if 'input_ids' in inputs else None}")
        self.debug_print(f"prepare_inputs_for_generation: attention_mask shape: {inputs['attention_mask'].shape if 'attention_mask' in inputs else None}")
        self.debug_print(f"prepare_inputs_for_generation: inputs_embeds shape: {inputs.get('inputs_embeds', {}).shape if inputs.get('inputs_embeds') is not None else None}")

        return inputs

    def get_input_embeddings(self):
        return self.phi3.get_input_embeddings()

    def set_input_embeddings(self, value):
        self.phi3.set_input_embeddings(value)

    def gradient_checkpointing_enable(self, **kwargs):
        self.phi3.gradient_checkpointing_enable(**kwargs)

    def gradient_checkpointing_disable(self):
        self.phi3.gradient_checkpointing_disable()

    def __getattr__(self, name):
        try:
            return super().__getattr__(name)
        except AttributeError:
            return getattr(self.phi3, name)

    def generate(self, input_ids=None, attention_mask=None, image_embeddings=None, **kwargs):
        if image_embeddings is not None:
            kwargs['image_embeddings'] = image_embeddings
            self.debug_print(f"generate input_ids: {input_ids.size()}")
            self.debug_print(f"generate image_embedding: {image_embeddings.size()}")

        if attention_mask is not None and image_embeddings is not None:
            # Add an extra attention mask token for the image embedding
            self.debug_print(f"generate before: {attention_mask.size()}")
            attention_mask = torch.cat([torch.ones(attention_mask.size(0), 1, device=attention_mask.device), attention_mask], dim=1)
            self.debug_print(f"generate after: {attention_mask.size()}")

        return super().generate(input_ids=input_ids, attention_mask=attention_mask, **kwargs)


In [ ]:
# Define the path in Google Drive where you want to save the checkpoints
gdrive_checkpoint_dir = "/content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints"

# Ensure the directory exists
os.makedirs(gdrive_checkpoint_dir, exist_ok=True)

class SaveLatestCheckpointAndLoraCallback(TrainerCallback):
    # def __init__(self, tokenizer):
    #     super().__init__()
    #     self.tokenizer = tokenizer

    def on_save(self, args, state, control, **kwargs):
        if state.is_world_process_zero:
            checkpoint_dir = os.path.join(gdrive_checkpoint_dir, f"checkpoint-{state.global_step}")
            os.makedirs(checkpoint_dir, exist_ok=True)

            # Save the model and tokenizer
            kwargs["model"].save_pretrained(checkpoint_dir)
            # Save the tokenizer
            kwargs["tokenizer"].save_pretrained(checkpoint_dir)
            # self.tokenizer.save_pretrained(checkpoint_dir)

            # Log and save the projector
            projector = kwargs["model"].base_model.model.projector
            weight_change = projector.get_weight_change()
            print(f"Projector weight change at step {state.global_step}: {weight_change}")

            # # Add gradient checking here
            # for name, param in projector.named_parameters():
            #     if param.grad is None:
            #         print(f"No gradient for {name}")
            #     else:
            #         print(f"Gradient norm for {name}: {param.grad.norm().item()}")

            projector_path = os.path.join(checkpoint_dir, "image_projector.pth")
            torch.save(projector.state_dict(), projector_path)

            # Save LoRA weights
            lora_state_dict = {}
            for name, module in kwargs["model"].base_model.model.phi3.named_modules():
                if isinstance(module, lora.Linear4bit):
                    if hasattr(module, 'lora_A'):
                        lora_state_dict[f"{name}.lora_A.weight"] = module.lora_A.default.weight.data.cpu()
                        lora_state_dict[f"{name}.lora_B.weight"] = module.lora_B.default.weight.data.cpu()
                        lora_state_dict[f"{name}.scaling"] = module.scaling

                    # Save lora_embedding_A and lora_embedding_B if they exist and are not empty
                    if hasattr(module, 'lora_embedding_A') and module.lora_embedding_A:
                        lora_state_dict[f"{name}.lora_embedding_A"] = {k: v.cpu() for k, v in module.lora_embedding_A.items()}
                    if hasattr(module, 'lora_embedding_B') and module.lora_embedding_B:
                        lora_state_dict[f"{name}.lora_embedding_B"] = {k: v.cpu() for k, v in module.lora_embedding_B.items()}

            torch.save(lora_state_dict, os.path.join(checkpoint_dir, "lora_weights.pt"))

            # Explicitly save the trainer state
            trainer_state_path = os.path.join(checkpoint_dir, "trainer_state.json")
            state_dict = dataclasses.asdict(state)
            with open(trainer_state_path, "w") as f:
                json.dump(state_dict, f, indent=2)

            # Remove previous checkpoint
            prev_checkpoint = os.path.join(gdrive_checkpoint_dir, f"checkpoint-{state.global_step - args.save_steps}")
            if os.path.exists(prev_checkpoint):
                import shutil
                shutil.rmtree(prev_checkpoint)

            print(f"Saved checkpoint to {checkpoint_dir}")

    #         # Upload the checkpoint to Hugging Face Hub
    #         self.upload_to_hub(checkpoint_dir, args.hub_model_id)

    # def upload_to_hub(self, checkpoint_dir, hub_model_id):
    #     api = HfApi()
    #     api.upload_folder(
    #         folder_path=checkpoint_dir,
    #         repo_id=hub_model_id,
    #         repo_type="model",
    #     )
    #     print(f"Uploaded checkpoint {checkpoint_dir} to Hugging Face Hub")

# Function to get the latest checkpoint
def get_latest_checkpoint(checkpoint_dir):
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]
    if not checkpoints:
        return None
    latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
    return os.path.join(checkpoint_dir, latest_checkpoint)

In [ ]:
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
#   attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
#   attn_implementation = 'sdpa'

# print(attn_implementation)
print(compute_dtype)

torch.bfloat16


In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# # Load the model again for quantization
# ### Download Phi-3 model
# phi3_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     trust_remote_code=True,
#     quantization_config=bnb_config,
#     device_map=device_map,
#     torch_dtype=compute_dtype,
#     # attn_implementation=attn_implementation
# )

# print(phi3_model)

In [ ]:
def load_lora_weights(checkpoint_path, device):
    # Check if the checkpoint_path is a local directory or a Hugging Face repo
    if os.path.isdir(checkpoint_path):
        # Local directory
        lora_weights_path = os.path.join(checkpoint_path, "lora_weights.pt")
        if os.path.exists(lora_weights_path):
            lora_state_dict = torch.load(lora_weights_path, map_location=device)
        else:
            raise FileNotFoundError(f"LoRA weights file not found in {checkpoint_path}")
    else:
        # Assume it's a Hugging Face repo
        try:
            # Try to download the file from the Hugging Face repo
            lora_weights_path = hf_hub_download(repo_id=checkpoint_path, filename="lora_weights.pt")
            lora_state_dict = torch.load(lora_weights_path, map_location=device)
        except Exception as e:
            raise ValueError(f"Error loading LoRA weights from Hugging Face repo: {str(e)}")

    return lora_state_dict

def load_model_with_lora_and_projector(checkpoint_path, device, bnb_config=None, debug=False, for_training=False):
    is_cpu = device == "cpu"

    common_kwargs = {
        "trust_remote_code": True,
        "debug": debug
    }

    if is_cpu:
        device_map = None
        model = Phi3WithProjector.from_pretrained(
            checkpoint_path,
            torch_dtype=torch.float32,
            low_cpu_mem_usage=True,
            device_map=device_map,
            **common_kwargs
        )
    else:
        device_map = {"": 0}  # This maps all modules to the specified device
        model = Phi3WithProjector.from_pretrained(
            checkpoint_path,
            quantization_config=bnb_config,
            device_map=device_map,  # Use the corrected device_map
            torch_dtype=torch.float16,
            attn_implementation='eager',
            **common_kwargs
        )

    model = model.to(device)

    # Load LoRA weights
    lora_state_dict = load_lora_weights(checkpoint_path, device)
    print(f"Total keys in lora_state_dict: {len(lora_state_dict)}")

    new_state_dict = {}
    scaling_factors_loaded = 0

    for name, module in model.phi3.named_modules():
        if isinstance(module, lora.Linear4bit):
            lora_A_key = f"{name}.lora_A.weight"
            lora_B_key = f"{name}.lora_B.weight"
            scaling_key = f"{name}.scaling"

            if lora_A_key in lora_state_dict and lora_B_key in lora_state_dict:
                new_state_dict[f"{name}.lora_A.default.weight"] = lora_state_dict[lora_A_key]
                new_state_dict[f"{name}.lora_B.default.weight"] = lora_state_dict[lora_B_key]

                if scaling_key in lora_state_dict:
                    module.scaling = lora_state_dict[scaling_key]
                    scaling_factors_loaded += 1
            else:
                print(f"Warning: LoRA weights for {name} not found in checkpoint")

    # Load the filtered state dict
    model.phi3.load_state_dict(new_state_dict, strict=False)

    print(f"Loaded LoRA weights: {len(new_state_dict)} / {len(lora_state_dict)}")
    print(f"Loaded scaling factors: {scaling_factors_loaded}")
    print(f"Total LoRA modules processed: {len(new_state_dict) // 2}")

    if not for_training:
        # Prepare the model for inference only if not for training
        for name, module in model.named_modules():
            if isinstance(module, LinearFP4): # or isinstance(module, LinearFP8):
                module.prepare_for_inference()
    else:
        # Ensure the model is in training mode
        model.train()

    return model

In [ ]:
# !cp -r /content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoint-350 /content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints/

In [ ]:

# Get the latest checkpoint
latest_checkpoint = get_latest_checkpoint(gdrive_checkpoint_dir)
eval_first = False
if latest_checkpoint:
    print(f"Loading model from checkpoint: {latest_checkpoint}")
    model = load_model_with_lora_and_projector(latest_checkpoint, device,
                                               bnb_config=bnb_config, for_training = True)
    eval_first = True
else:
    print("No checkpoint found. Starting training from scratch.")
    phi3_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        quantization_config=bnb_config,
        device_map=device_map,
        torch_dtype=compute_dtype,
        attn_implementation='eager'
    )
    image_embedding_dim = len(hf_dataset[0]['image_embedding'])
    projection_dim = phi3_model.config.hidden_size
    projector = ImageProjector(image_embedding_dim, projection_dim).to(device)
    model = Phi3WithProjector(phi3_model, projector)

    # Prepare the model for k-bit training
    model = prepare_model_for_kbit_training(model)

Loading model from checkpoint: /content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints/checkpoint-350


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading adapter weights from /content/drive/MyDrive/multimodal_llm/phi-3_5/checkpoints/checkpoint-350 led to unexpected keys not found in the model:  ['phi3.model.layers.0.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.0.mlp.down_proj.lora_B.default.weight', 'phi3.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'phi3.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'phi3.model.layers.1.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.1.mlp.down_proj.lora_B.default.weight', 'phi3.model.layers.1.self_attn.o_proj.lora_A.default.weight', 'phi3.model.layers.1.self_attn.o_proj.lora_B.default.weight', 'phi3.model.layers.10.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.10.mlp.down_proj.lora_B.default.weight', 'phi3.model.layers.10.self_attn.o_proj.lora_A.default.weight', 'phi3.model.layers.10.self_attn.o_proj.lora_B.default.weight', 'phi3.model.layers.11.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.11.mlp.down_proj.lora_B.default.weight',

UnboundLocalError: local variable 'target_dtype' referenced before assignment

In [ ]:
print(compute_dtype) , print(model)

torch.float16
Phi3WithProjector(
  (phi3): Phi3ForCausalLM(
    (model): Phi3Model(
      (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
      (embed_dropout): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0-31): 32 x Phi3DecoderLayer(
          (self_attn): Phi3Attention(
            (o_proj): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=3072, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=3072, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lora_magnitude_vector): ModuleDict()
            )
            (qkv_pr

(None, None)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 8912896 || all params: 2021727232 || trainable%: 0.44085551497384196


In [ ]:
def verify_projector_weight_changes(loaded_state_dict, initial_state_dict):
    for key in loaded_state_dict:
        if torch.norm(loaded_state_dict[key] - initial_state_dict[key]).item() > 0:
            print(f"Weights changed for {key}")
            return True
    print("WARNING: No weight changes detected in the projector!")
    return False

# Use this when loading the model
initial_projector_state = copy.deepcopy(model.base_model.model.projector.state_dict())

### Training

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size  = per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    eval_strategy="steps",
    eval_steps=eval_steps, # Evaluate every 25 steps
    # Add these new arguments
    do_eval=True,
    eval_delay=0,  # Start evaluation immediately
    # Enable gradient checkpointing
    gradient_checkpointing=gradient_checkpointing,
    # Disable data parallelism if not needed
    ddp_find_unused_parameters=False,
    save_total_limit=1,  # Keep only the latest checkpoint
    hub_model_id = hf_adapter_repo,
    push_to_hub=False,  # We'll handle this in our custom callback
)



In [ ]:
# Custom data collator to handle pre-tokenized inputs
def custom_data_collator(features):
    batch = {k: [d[k] for d in features] for k in features[0].keys()}

    # Stack image embeddings
    batch['image_embeddings'] = torch.stack(batch['image_embeddings'])#.to(torch.float16)

    # Pad the sequences
    batch['input_ids'] = torch.nn.utils.rnn.pad_sequence(batch['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id)
    batch['attention_mask'] = torch.nn.utils.rnn.pad_sequence(batch['attention_mask'], batch_first=True, padding_value=0)
    batch['labels'] = torch.nn.utils.rnn.pad_sequence(batch['labels'], batch_first=True, padding_value=-100)

    return batch

# Function to select a random subset of the dataset
def select_subset(dataset, fraction=0.05):
    num_samples = int(len(dataset) * fraction)
    indices = random.sample(range(len(dataset)), num_samples)
    return dataset.select(indices)

# Select 5% of the training and test datasets
small_train_dataset = select_subset(dataset_dict['train'], fraction=0.2)
small_test_dataset = select_subset(dataset_dict['test'], fraction=0.05)

# Create a new DatasetDict with the smaller datasets
small_dataset_dict = DatasetDict({
    'train': small_train_dataset,
    'test': small_test_dataset
})

print(f"Small train dataset size: {len(small_dataset_dict['train'])}")
print(f"Small test dataset size: {len(small_dataset_dict['test'])}")

Small train dataset size: 8626
Small test dataset size: 113


In [ ]:
# Before initializing the SFTTrainer
model.projector.train()  # Set projector to training mode
for param in model.projector.parameters():
    param.requires_grad = True

def hook_fn(module, grad_input, grad_output):
    print(f"Gradient for {module.__class__.__name__}:")
    for idx, grad in enumerate(grad_input):
        if grad is not None:
            print(f"  Input gradient {idx}: {grad.norm().item()}")

# # Apply the hook to projector layers
# model.projector.layer1.register_backward_hook(hook_fn)
# model.projector.layer2.register_backward_hook(hook_fn)

In [ ]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    # eval_dataset=dataset_dict['test'],
    # train_dataset=small_dataset_dict['train'],
    eval_dataset=small_dataset_dict['test'],
    tokenizer=tokenizer,
    data_collator=custom_data_collator,
    peft_config=lora_config,
    max_seq_length=max_seq_length,
    packing=packing,
    # callbacks=[SaveLatestCheckpointCallback(), SaveQLoraAdapterCallback(model)],  # Add the custom callback
    callbacks=[SaveLatestCheckpointAndLoraCallback()],
)
# # Cast the model's projector to float32 before evaluation
# model.projector = model.projector.to(torch.float32)
# Perform initial evaluation
if eval_first:
    print("Performing initial evaluation...")
    # eval_results = trainer.evaluate()
    # print(f"Initial evaluation results: {eval_results}")
    # Start or resume training
    trainer.train(resume_from_checkpoint=latest_checkpoint)
else:
    # Start training
    trainer.train()


Performing initial evaluation...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
	eval_steps: 5 (from args) != 50 (from trainer_state.js

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# # Save the fine-tuned model
# trainer.save_model()
# # trainer.model.save_pretrained(new_model)
# final_model_path = os.path.join(gdrive_checkpoint_dir, "final_model")
# trainer.model.save_pretrained(final_model_path)
# tokenizer.save_pretrained(final_model_path)

In [ ]:
# torch.save(model.base_model.model.projector.state_dict(), final_model_path + '/image_projector.pth')
# print(f"Projector saved to: {final_model_path}/image_projector.pth")

In [ ]:
# # Save the projector
# projector_path = '/content/drive/MyDrive/multimodal_llm/phi-3_5/image_projector.pth'
# os.makedirs(os.path.dirname(projector_path), exist_ok=True)
# torch.save(model.projector.state_dict(), projector_path)
# print(f"Projector saved to: {projector_path}")

In [ ]:
# Get the latest checkpoint
latest_checkpoint = get_latest_checkpoint(gdrive_checkpoint_dir)
loaded_projector_state = torch.load(latest_checkpoint + '/image_projector.pth')
verify_projector_weight_changes(loaded_projector_state, initial_projector_state)

<ipython-input-20-aa93cb025ceb>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_projector_state = torch.load(latest_checkpoint + '/image_projector.pth')


NameError: name 'verify_projector_weight_changes' is not defined

In [ ]:
# trainer.push_to_hub()
api = HfApi()
api.upload_folder(
    folder_path=latest_checkpoint,
    repo_id=hf_adapter_repo,
    repo_type="model",
)

CommitInfo(commit_url='https://huggingface.co/sayanbanerjee32/multimodal-phi3_5-mini-instruct-llava_adapter/commit/4b0b5dec7ec64d6a7bc4dc98b6dd559ea415e094', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4b0b5dec7ec64d6a7bc4dc98b6dd559ea415e094', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# !cp -r multimodal-phi3_5-mini-instruct-llava_adapter /content/drive/MyDrive/multimodal_llm/phi-3_5

In [ ]:
# Add this function after your CustomTextGenerator class
def evaluate_model(model, tokenizer, eval_dataset, device):
    model = model.to(device).to(torch.float16)
    model.eval()
    total_loss = 0
    num_batches = 0

    dataloader = DataLoader(eval_dataset, batch_size=2, collate_fn=custom_data_collator)

    with torch.no_grad():
        for batch in dataloader:
            batch = {
                'input_ids': batch['input_ids'].to(device).long(),
                'attention_mask': batch['attention_mask'].to(device).long(),
                'image_embeddings': batch['image_embeddings'].to(device).to(torch.float16),
                'labels': batch['labels'].to(device).long()
            }

            # print(f"input_ids shape: {batch['input_ids'].shape}, dtype: {batch['input_ids'].dtype}")
            # print(f"attention_mask shape: {batch['attention_mask'].shape}, dtype: {batch['attention_mask'].dtype}")
            # print(f"image_embeddings shape: {batch['image_embeddings'].shape}, dtype: {batch['image_embeddings'].dtype}")
            # print(f"labels shape: {batch['labels'].shape}, dtype: {batch['labels'].dtype}")

            try:
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    image_embeddings=batch['image_embeddings'],
                    labels=batch['labels']
                )
                total_loss += outputs.loss.item()
                num_batches += 1
            except RuntimeError as e:
                print(f"Error processing batch: {e}")
                print(f"Model device: {next(model.parameters()).device}")
                for name, param in model.named_parameters():
                    print(f"{name} - shape: {param.shape}, dtype: {param.dtype}, device: {param.device}")
                raise  # Re-raise the exception to see the full traceback

    return total_loss / num_batches if num_batches > 0 else 0

# Add these lines before merging the model
print("Evaluating model before merging...")
pre_merge_loss = evaluate_model(model, tokenizer, small_dataset_dict['test'], device)
print(f"Pre-merge loss: {pre_merge_loss}")

Evaluating model before merging...
Pre-merge loss: nan


## sample inference code

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Create a custom text generation class
class CustomTextGenerator:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def generate(self, input_text, image_embedding, **generate_kwargs):
        # Tokenize the input text
        inputs = self.tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs["input_ids"].to(self.model.device)
        attention_mask = inputs["attention_mask"].to(self.model.device)

        # Ensure image_embedding is a tensor and move it to the correct device
        if not isinstance(image_embedding, torch.Tensor):
            image_embedding = torch.tensor(image_embedding)
        # image_embedding = image_embedding.to(self.model.device).unsqueeze(0)  # Add batch dimension
        image_embedding = image_embedding.to(self.model.device)#.to(next(self.model.parameters()).dtype)

        # Generate text
        outputs = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # image_embeddings=image_embedding,
            image_embeddings=image_embedding.unsqueeze(0),  # Add batch dimension
            **generate_kwargs
        )

        # Decode the generated text
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return generated_text

In [ ]:
# Get a sample from the validation set
sample = dataset_dict['test'][1]
image_embedding = sample['image_embeddings']


def get_first_user_input(decoded_text):
    # Find the position of the first <|assistant|> tag
    assistant_pos = decoded_text.find('<|assistant|>')

    # If <|assistant|> is found, truncate the text
    if assistant_pos != -1:
        return decoded_text[:assistant_pos].strip()
    else:
        return decoded_text.strip()

# Decode the input_ids
full_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=False)

# Extract only the first user input
input_text = get_first_user_input(full_text) + '<|assistant|>'
print(input_text)

<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>

In [ ]:
# Generate text
# Initialize the custom text generator
generator = CustomTextGenerator(model=model, tokenizer=tokenizer)
generated_text = generator.generate(
    input_text,
    image_embedding=image_embedding,
    # max_length=200,
    # num_return_sequences=1,
    # do_sample=True,
    # temperature=0.7,
    # top_k=50,
    # top_p=0.95,
    max_new_tokens=150,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.8,
    top_k=40,
    top_p=0.9,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
)

print("Input text:")
print(input_text)
print("\nGenerated text:")
print(generated_text)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def test_inference(model, tokenizer, image_embedding):
    # Log projector weights before inference
    projector = model.projector
    weight_change = projector.get_weight_change()
    print(f"Projector weight change: {weight_change}")
    # Ensure the model is in the correct precision (e.g., float16)
    # model = model.to(next(model.parameters()).dtype)

    generator = CustomTextGenerator(model=model, tokenizer=tokenizer)
    input_text = """<|system|> You are a helpful assistant.<|end|><|user|> Given the following information, provide a detailed and accurate response:
Describe the image in detail.
<|end|><|assistant|>
"""
# [An image is provided for this task.]
    generated_text = generator.generate(
        input_text,
        image_embedding=image_embedding,
        max_new_tokens=150,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.8,
        top_k=40,
        top_p=0.9,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
    )
    return generated_text



In [ ]:
# Before merging
print("Testing inference before merging...")
pre_merge_output = test_inference(model, tokenizer, sample['image_embeddings'])
print(pre_merge_output)


Testing inference before merging...


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### merge models and save in gdrive

In [ ]:
del trainer
del model
gc.collect()
torch.cuda.empty_cache()

NameError: name 'trainer' is not defined

In [ ]:
loaded_model = load_model_with_lora_and_projector(hf_adapter_repo, #latest_checkpoint,
                                                  device, bnb_config = bnb_config, debug = False)
# print(loaded_model.projector.layer1.weight.dtype, loaded_model.projector.layer1.bias.dtype)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading adapter weights from sayanbanerjee32/multimodal-phi3_5-mini-instruct-llava_adapter led to unexpected keys not found in the model:  ['phi3.model.layers.0.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.0.mlp.down_proj.lora_B.default.weight', 'phi3.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'phi3.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'phi3.model.layers.1.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.1.mlp.down_proj.lora_B.default.weight', 'phi3.model.layers.1.self_attn.o_proj.lora_A.default.weight', 'phi3.model.layers.1.self_attn.o_proj.lora_B.default.weight', 'phi3.model.layers.10.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.10.mlp.down_proj.lora_B.default.weight', 'phi3.model.layers.10.self_attn.o_proj.lora_A.default.weight', 'phi3.model.layers.10.self_attn.o_proj.lora_B.default.weight', 'phi3.model.layers.11.mlp.down_proj.lora_A.default.weight', 'phi3.model.layers.11.mlp.down_proj.lora_B.default.weight', 'phi3.mode

Loaded projector with input_dim=512, output_dim=3072


<ipython-input-18-153e88ecfc4b>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_state_dict = torch.load(lora_weights_path, map_location=device)


Total keys in lora_state_dict: 192
Loaded LoRA weights: 128 / 192
Loaded scaling factors: 64
Total LoRA modules processed: 64


In [ ]:
def evaluate_loaded_model(model, tokenizer, eval_dataset, device):
    model = model.to(device)
    model.eval()

    # # Print model parameter dtypes
    # for name, param in model.named_parameters():
    #     if 'projector' in name:
    #         print(f"{name} dtype: {param.dtype}")
    total_loss = 0
    num_batches = 0

    dataloader = DataLoader(eval_dataset, batch_size=2, collate_fn=custom_data_collator)

    with torch.no_grad():
        for batch in dataloader:
            batch = {
                'input_ids': batch['input_ids'].to(device),
                'attention_mask': batch['attention_mask'].to(device),
                'image_embeddings': batch['image_embeddings'].to(device),#.to(torch.float16),
                'labels': batch['labels'].to(device)
            }

            # Print the dtype of image_embeddings
            # print(f"image_embeddings dtype: {batch['image_embeddings'].dtype}")

            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                image_embeddings=batch['image_embeddings'],
                labels=batch['labels']
            )
            total_loss += outputs.loss.item()
            num_batches += 1

    return total_loss / num_batches if num_batches > 0 else 0

print("Evaluating loaded model...")
loded_model_loss = evaluate_loaded_model(loaded_model, tokenizer, small_dataset_dict['test'], device)
print(f"loded model loss: {loded_model_loss}")

Evaluating loaded model...


<ipython-input-31-e3339eabcc31>:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


loded model loss: 0.18099397244422058


In [ ]:
# after loading model from PEFT adapter
print("Testing inference after loading model from peft adapter...")
peft_adapter_output = test_inference(loaded_model, tokenizer, sample['image_embeddings'])
print(peft_adapter_output)

Testing inference after loading model from peft adapter...
Projector weight change: 2.8105183839797974


<ipython-input-31-e3339eabcc31>:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


You are a helpful assistant. Given the following information, provide a detailed and accurate response:
Describe the image in detail.
 <|Animal-Based Dining Experience with an Eye for Detail!<\|end\|> 
In this snapshot of dine aloud's menu page on TripAdvisor® travel reviews website under "What to do & see", there is much more than just food options available; it captures your attention through multiple visual elements that create intrigue about their establishment which offers animal meals as well other specialty items like pizza or beef steak burgers among others - making sure visitors feel invited right away even before entering into reading further details such...   
How does one get these delicious offerings?   What kind would you choose from them if given free reign over ordering at any time


In [ ]:
print('end')